1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [10]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

import pandas as pd
import numpy as np

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [11]:
file_name = "./data_000637.txt"
data = pd.read_csv(file_name)

if data.shape[0] <= 10000: N = data.shape[0]
else: N = int(np.random.randint(10000, data.shape[0], 1))

data_N_rows = pd.read_csv(file_name, nrows=N)
print(f"Number of rows read: {N}") 
print(data_N_rows)

Number of rows read: 882488
        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0          1     0          123  3869200167        2374        26
1          1     0          124  3869200167        2374        27
2          1     0           63  3869200167        2553        28
3          1     0           64  3869200167        2558        19
4          1     0           64  3869200167        2760        25
...      ...   ...          ...         ...         ...       ...
882483     1     0           58  3869208225         272        23
882484     1     0           60  3869208225         276         0
882485     1     0           57  3869208225         282        11
882486     1     1            8  3869208225         284        17
882487     1     0           31  3869208225         941        13

[882488 rows x 6 columns]


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [12]:

x = np.max(data['BX_COUNTER']) 
print(f"Estimated number of BX in an ORBIT: {x}")


Estimated number of BX in an ORBIT: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [13]:

TDC_per_ns = 25 / 30 
bx_counter_ns = 25
data['TDC_TIME_ns'] = data['TDC_MEAS'] * TDC_per_ns
data['BX_TIME_ns'] = data['BX_COUNTER'] * bx_counter_ns
data['ORBIT_TIME_ns'] = data['ORBIT_CNT'] * x * bx_counter_ns
data['AbsoluteTime_ns'] = data['TDC_TIME_ns'] + data['BX_TIME_ns'] + data['ORBIT_TIME_ns']
data['AbsoluteTime_ns'] -= data['AbsoluteTime_ns'].iloc[0] #introducing the offset
data['TimeSeries'] = pd.to_timedelta(data['AbsoluteTime_ns'], unit='ns')

print(data[['TDC_TIME_ns', 'BX_TIME_ns', 'ORBIT_TIME_ns', 'AbsoluteTime_ns', 'TimeSeries']].head())

   TDC_TIME_ns  BX_TIME_ns    ORBIT_TIME_ns  AbsoluteTime_ns  \
0    21.666667       59350  344649004875525           0.0000   
1    22.500000       59350  344649004875525           0.8125   
2    23.333333       63825  344649004875525        4476.6250   
3    15.833333       63950  344649004875525        4594.1250   
4    20.833333       69000  344649004875525        9649.1250   

                 TimeSeries  
0           0 days 00:00:00  
1           0 days 00:00:00  
2 0 days 00:00:00.000004476  
3 0 days 00:00:00.000004594  
4 0 days 00:00:00.000009649  


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [14]:
duration = data['TimeSeries'].iloc[-1] # since data['TimeSeries'].iloc[0] is set to zero, if no offset duration = data['TimeSeries'].iloc[-1] - data['TimeSeries'].iloc[0]
print(f"Duration of data taking: {duration.components.hours} hours, {duration.components.minutes} minutes, {duration.components.seconds} seconds and {duration.components.milliseconds} milliseconds")

Duration of data taking: 0 hours, 0 minutes, 0 seconds and 980 milliseconds


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [15]:
grouped = data.groupby('TDC_CHANNEL')['ORBIT_CNT'].sum()
sorted_grouped = grouped.sort_values(ascending=False)
top3 = sorted_grouped.head(3)
print(top3)

TDC_CHANNEL
139    418102549486957
64     255444994368975
63     250113227630194
Name: ORBIT_CNT, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [16]:
non_empty_orbits = data['ORBIT_CNT'].count()
print(f"Number of non empty orbits: {non_empty_orbits} ")

Number of non empty orbits: 1310720 


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [17]:
TDC_channel_139 = data[data['TDC_CHANNEL'] == 139]
unique_orbits_tdc_139 = TDC_channel_139['ORBIT_CNT'].nunique()
print(f"Number of unique orbits for TDC_CHANNEL=139: {unique_orbits_tdc_139}")

Number of unique orbits for TDC_CHANNEL=139: 10976


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [18]:
fpga_0 = data[data['FPGA'] == 0]
ser_fpga_0 = pd.Series(fpga_0['ORBIT_CNT'], fpga_0['TDC_CHANNEL'])
fpga_1 = data[data['FPGA'] == 1]
ser_fpga_1 = pd.Series(fpga_1['ORBIT_CNT'], fpga_1['TDC_CHANNEL'])

print(f"For FPGA 0: \n {ser_fpga_0}")
print(f"For FPGA 1: \n {ser_fpga_1}")

For FPGA 0: 
 TDC_CHANNEL
123             NaN
124    3.869200e+09
63              NaN
64     3.869200e+09
64     3.869200e+09
           ...     
60              NaN
62              NaN
64     3.869200e+09
139    3.869200e+09
61              NaN
Name: ORBIT_CNT, Length: 922852, dtype: float64
For FPGA 1: 
 TDC_CHANNEL
7     NaN
6     NaN
139   NaN
8     NaN
5     NaN
       ..
38    NaN
36    NaN
39    NaN
35    NaN
4     NaN
Name: ORBIT_CNT, Length: 387868, dtype: float64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.